In [ ]:
!pip install ipywidgets
from google.colab import output
output.enable_custom_widget_manager()

import pandas as pd
import os
import re
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.express as px
%matplotlib inline
import matplotlib.pyplot as plt
#plt.ion()  # Enable interactive mode

# Set display options-
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)


In [ ]:

def create_time_frame(from_year = 2024, to_year = 2024, from_month = 8, to_month = 8, from_date = 1, to_date = 11):
    # Tạo khung thời gian
    from datetime import datetime, timedelta, date

    def date_range(start, end):
        delta = end - start  # as timedelta
        days = [start + timedelta(days=day) for day in range(delta.days + 1)]
        return days
    start = date(from_year, from_month, from_date)
    end = date(to_year, to_month, to_date)
    from_to_date = date_range(start, end)
    from_to_date = set(from_to_date)
    return from_to_date

# def create_plot_general(df, feature_list: list, chart_name: str):
#     def create_plot(features):
#         plt.close('all')  # Close any existing plots
#         fig, ax = plt.subplots(figsize=(8, 2.5))
        
#         # Convert features to a list if it's a single string
#         if isinstance(features, str):
#             features = [features]
        
#         # Create a combined string for multi-column grouping
#         if len(features) > 1:
#             df['combined'] = df[features].apply(lambda row: ' - '.join(row.values.astype(str)), axis=1)
#             plot_data = df['combined'].value_counts()
#         else:
#             plot_data = df[features[0]].value_counts()
    
#         try:
#             plot_data.plot(kind='barh', color='skyblue', ax=ax)
#             ax.set_xlabel('Số lượng')
#             ax.set_ylabel('')
#             ax.set_title(f'{chart_name} - {", ".join(features)}')
#             ax.invert_yaxis()
#             ax.grid(axis='x', linestyle='--', linewidth=0.7, color='grey')
#             plt.tight_layout()
            
#             # Adjust figure size based on the number of categories
#             fig.set_size_inches(10, max(6, len(plot_data) * 0.4))
            
#             # Display value counts
#             value_counts = plot_data.reset_index()
#             value_counts.columns = [' - '.join(features), 'Count']
#             return fig, value_counts
#         except IndexError:
#             pass
            
#     output = widgets.Output()
    
#     def on_button_clicked(b):
#         with output:
#             clear_output(wait=True)
#             features = eval(b.description)  # Convert string representation of list to actual list
#             try:
#                 fig, value_counts = create_plot(features)
#                 display(fig)
#                 display(value_counts)
#             except TypeError:
#                 print('Mục này không có dữ liệu, nhập nhiều vô mà phân tích nhé 😆😆😆🫡🫡')
    
#     # Create buttons with feature combinations
#     button_features = feature_list
    
#     buttons = [widgets.Button(description=str(feature)) for feature in button_features]
    
#     # Set on_click event for each button
#     for button in buttons:
#         button.on_click(on_button_clicked)
    
#     # Display buttons and output area
#     display(widgets.HBox(buttons))
#     display(output)


def create_plot_general(df, feature_list: list, chart_name: str):
    def create_plots(features):
        plt.close('all')  # Close any existing plots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))
        
        # Convert features to a list if it's a single string
        if isinstance(features, str):
            features = [features]
        
        # Create a combined string for multi-column grouping
        if len(features) > 1:
            df['combined'] = df[features].apply(lambda row: ' - '.join(row.values.astype(str)), axis=1)
            plot_data = df['combined'].value_counts()
        else:
            plot_data = df[features[0]].value_counts()
    
        try:
            # Bar Chart
            plot_data.plot(kind='barh', color='skyblue', ax=ax1)
            ax1.set_xlabel('Số lượng')
            ax1.set_ylabel('')
            ax1.set_title(f'Bar Chart: {chart_name} - {", ".join(features)}')
            ax1.invert_yaxis()
            ax1.grid(axis='x', linestyle='--', linewidth=0.7, color='grey')
            
            # Pie Chart
            plot_data.plot(kind='pie', ax=ax2, autopct='%1.1f%%', startangle=90)
            ax2.set_ylabel('')
            ax2.set_title(f'Pie Chart: {chart_name} - {", ".join(features)}')
            
            plt.tight_layout()
            
            # Adjust figure size based on the number of categories
            fig.set_size_inches(16, max(5, len(plot_data) * 0.4))
            
            # Display value counts
            value_counts = plot_data.reset_index()
            value_counts.columns = [' - '.join(features), 'Count']
            return fig, value_counts
        except IndexError:
            pass
            
    output = widgets.Output()
    
    def on_button_clicked(b):
        with output:
            clear_output(wait=True)
            features = eval(b.description)  # Convert string representation of list to actual list
            try:
                fig, value_counts = create_plots(features)
                display(fig)
                display(value_counts)
            except TypeError:
                print('Mục này không có dữ liệu, nhập nhiều vô mà phân tích nhé 😆😆😆🫡🫡')
    
    # Create buttons with feature combinations
    button_features = feature_list
    
    buttons = [widgets.Button(description=str(feature)) for feature in button_features]
    
    # Set on_click event for each button
    for button in buttons:
        button.on_click(on_button_clicked)
    
    # Display buttons and output area
    display(widgets.HBox(buttons))
    display(output)


# Đọc dữ liệu GPS checkin shop
def read_gps(list_report):
    list_gps = []
    for report in list_report:
        if 'GPS' in report:
            file_path = os.path.join(base_path, report)
            df_gps = pd.read_excel(file_path, skiprows=0)
            try:
                # Bởi vì dòng số 1 trong file dữ liệu GPS không có dữ liệu nên bỏ đi
                df_gps = df_gps.drop(df_gps.index[0])
            except IndexError:
                pass
            list_gps.append(df_gps)
    df_gps = pd.concat(list_gps, ignore_index=True)

    # Chuyển đổi tên cột Date sang Day
    df_gps['Day'] = df_gps['Date']
    # Tạo cột Apple ID
    df_gps['Apple ID'] = pd.to_numeric(df_gps['Apple Store ID'], errors='coerce').fillna(0).astype(int)
    # Tạo cột Date để check ngày
    df_gps['Date'] = pd.to_datetime(df_gps[['Year', 'Month', 'Day']].astype('int'))
    # Xếp lại dữ liệu theo ngày nào mới nhất đưa lên đầu
    df_gps.sort_values('Date', inplace=True, ascending=False)
    # loại bỏ dữ liệu bị duplicate
    df_gps.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    # Bỏ qua các lượt checkin Apple Moment
    df_gps = df_gps[df_gps['Channel'] != 'A.M']
    # Chuyển đổi dữ liệu sai lệch check in sang định dạng int
    df_gps['Sai lệch'] = df_gps['Sai lệch'].map(lambda x: int(re.sub('[a-z\s,]', '', x)))
    
    # df_gps.reset_index(inplace=True)
    return df_gps



def read_sell_out(list_report):
    list_daily_sell = []
    for report in list_report:
        if 'Daily-Sell-out' in report:
            file_path = os.path.join(base_path, report)
            df_sell = pd.read_excel(file_path, sheet_name='Daily Sell - out', skiprows=1)
            try:
                df_sell = df_sell.drop(df_sell.index[0])
            except IndexError:
                pass
            list_daily_sell.append(df_sell)
    df_sell = pd.concat(list_daily_sell, ignore_index=True)

    # Chuyển đổi Date thành Day
    df_sell['Day'] = df_sell['Date']
    # Chuyển đổi cột Apple ID sang định dạng int
    df_sell['Apple ID'] = pd.to_numeric(df_sell['Apple ID'], errors='coerce').fillna(0).astype(int)
    # Tạo cột Date để lọc ngày
    df_sell['Date'] = pd.to_datetime(df_sell[['Year', 'Month', 'Day']].astype('int'))
    df_sell.sort_values('Date', inplace=True, ascending=False)
    # loại bỏ dữ liệu trùng lặp
    df_sell.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    
    return df_sell


# Đọc dữ liệu trong note
def read_daily_note(list_report):
    list_daily_note = []
    for report in list_report:
        if 'Note' in report:
            file_path = os.path.join(base_path, report)
            df_note = pd.read_excel(file_path, skiprows=1)
            list_daily_note.append(df_note)
    df_note = pd.concat(list_daily_note, ignore_index=True)
    # Chuyển đổi cột Apple ID sang định dạng int
    df_note['Apple ID'] = pd.to_numeric(df_note['Apple ID'], errors='coerce').fillna(0).astype(int)
    # Loại bỏ dữ liệu Apple Moment
    df_note = df_note[df_note['Channel'] != 'A.M']
    # Tạo cột Date để check khung thời gian
    df_note['Date'] = pd.to_datetime(df_note[['Year', 'Month', 'Day']].astype('int'))
    # Sắp xếp dữ liệu theo trình tự ngày gần nhất đưa lên đầu
    df_note.sort_values('Date', inplace=True, ascending=False)
    # loại bỏ dữ liệu trùng lắp nếu có, giữ lại ngày gần nhất
    df_note.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    return df_note


def read_oos(list_report):
    list_daily_oos = []
    for report in list_report:
        if 'OOS' in report:
            file_path = os.path.join(base_path, report)
    
            # Read the first 4 rows to handle headers manually
            df_header = pd.read_excel(file_path, header=None, nrows=4)
            header_1_16 = df_header.iloc[1, :16].tolist()
            header_17_20 = df_header.iloc[2, 16:20].tolist()
            header_21_80 = df_header.iloc[3, 20:81].tolist()
            header = header_1_16 + header_17_20 + header_21_80
            
            # Read the full data, skipping the first 4 rows (headers)
            df_oos = pd.read_excel(file_path, skiprows=3)
            df_oos = df_oos.iloc[:, :81]
            df_oos.columns = header
            list_daily_oos.append(df_oos)
    df_oos = pd.concat(list_daily_oos, ignore_index=True)
    # Chuyển đổi cột Apple ID sang định dạng int
    df_oos['Apple ID'] = pd.to_numeric(df_oos['Apple ID'], errors='coerce').fillna(0).astype(int)
    # Tạo cột Date để lọc ngày
    df_oos['Date'] = pd.to_datetime(df_oos[['Year', 'Month', 'Day']].astype('int'))
    # Xếp lại dữ liệu theo trình tự thời gian mới nhất
    df_oos.sort_values('Date', inplace=True, ascending=False)
    # Loại bỏ dữ liệu lặp lại nếu có gây ra bởi việc download nhầm nhiều bảng dữ liệu
    df_oos.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    # Move 'Date' to the 2nd column position
    cols = df_oos.columns.tolist()  # Get current column order
    cols.remove('Date')  # Remove 'Date' from the list
    cols.insert(1, 'Date')  # Insert 'Date' at the 2nd position (index 1)
    df_oos = df_oos[cols]
    return df_oos

# Bảng này là để tính toán dữ liệu và đưa vào bảng show oos=
def calcu_oos(df_oos_dropped_duplicates):
    # Define the column to start iterating after
    start_col = 'AW'
    
    # Find the index of the column to start from
    start_index = df_oos_dropped_duplicates.columns.get_loc(start_col) + 1
    
    # Iterate through the columns after the specified column
    counts_dict = {}
    for col in df_oos_dropped_duplicates.columns[start_index:]:
        try:
            count = (df_oos_dropped_duplicates[col] != 0).sum()
            counts_dict[col] = count
        except TypeError:
            print(df_oos_dropped_duplicates[col])
    
    # Convert the results to a DataFrame
    df = pd.DataFrame.from_dict(counts_dict, orient='index', columns=['Non-zero Count'])
    df['Percentage'] = round(df['Non-zero Count'] / len(df_oos_dropped_duplicates) * 100, 2)
    
    # Reset index to have the column names as a column
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Product'}, inplace=True)
    df.sort_values('Percentage', inplace=True)
    return df


def graph_showcase_oos(df):
    
    def update_graph(selected_range):
        # Filter the DataFrame based on the selected range
        if selected_range == '< 25%':
            filtered_df = df[df['Percentage'] < 25]
        elif selected_range == '25-50%':
            filtered_df = df[(df['Percentage'] >= 25) & (df['Percentage'] < 50)]
        elif selected_range == '50-75%':
            filtered_df = df[(df['Percentage'] >= 50) & (df['Percentage'] < 75)]
        elif selected_range == '> 75%':
            filtered_df = df[df['Percentage'] >= 75]
        
        # Create the bar chart
        fig = px.bar(filtered_df, x='Product', y='Percentage', 
                     title=f'Product Performance ({selected_range})',
                     labels={'Product': 'Product', 'Percentage': 'Percentage (%)'},
                     color='Percentage',
                     color_continuous_scale='Viridis')
        
        # Update layout for better readability
        fig.update_layout(
            xaxis_tickangle=-45,
            height=600,
            width=1000,
            xaxis_title='Tên sản phẩm',
            yaxis_title='Độ phủ của sản phẩm trên tổng số shop ASM visit(%)',
            coloraxis_colorbar=dict(title='Phần trăm (%)')
        )
        
        return fig
    
    # Create dropdown widget
    dropdown = widgets.Dropdown(
        options=[('< 25%', '< 25%'), ('25-50%', '25-50%'), ('50-75%', '50-75%'), ('> 75%', '> 75%')],
        value='< 25%',
        description='Percentage Range:',
        style={'description_width': 'initial'}
    )
    
    # Create output widget
    output = widgets.Output()
    
    # Display function
    def on_change(change):
        with output:
            output.clear_output(wait=True)
            fig = update_graph(change.new)
            display(fig)
    
    # Set up the callback
    dropdown.observe(on_change, names='value')
    
    # Display the dropdown and initial plot
    display(dropdown, output)
    
    # Show initial plot
    with output:
        fig = update_graph('< 25%')
        display(fig)

def who_doesnot_do_stock():
    if len(df_gps) != len(df_oos):
        # Merge DataFrames with indicator=True
        df_gps_to_check_stock = df_gps.drop_duplicates('Apple ID', keep='first')
    
        merged_df = df_gps_to_check_stock.merge(df_oos, on=['Staff Name', 'Apple ID', 'Channel', 'Store Name', 'Date'], 
                          how='left', indicator=True)
        
        # Filter rows that are only in df1 but not in df2
        df_gps_only = merged_df[merged_df['_merge'] == 'left_only'].drop(columns='_merge')
        df_gps_only =  df_gps_only[['Staff Name', 'Apple ID', 'Channel', 'Store Name', 'Date']]
    df_gps_only.reset_index()
    return df_gps_only

In [ ]:
from IPython.display import HTML, display, Javascript, clear_output
import matplotlib.pyplot as plt
import time
from PIL import Image

!wget -O Bruce_signature.png 'https://drive.google.com/uc?id=1baK9q3hzG55CHEkkBcucCNLERdHxG2RE'

time.sleep(1)
clear_output(wait=True)

image_path = 'Bruce_signature.png'
img = Image.open(image_path)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')
ax2.imshow(img)
plt.tight_layout()
plt.show()